# Lecture 8: Numerical Integration (odeint)

## CHME 7340 - Summer 2020

This notebook goes through the steps involved in numerically integrating ordinary differential equations that we will see in this course that are are tricky (or impossible) to solve analytically. We use the built-in ODE solver from the scipy package, called **odeint**. The main advantage of this approach is that, unlike in Euler's method, which uses fixed time steps, odeint automatically adjusts the time steps. This is useful in stiff problems that have multiple different time scales. Enzymatic conversions are a good example of this, where in the first part of the reaction where  𝑆>>𝐾𝑚  conversion is linear with time and we can take large time steps, but when the substrate begins to run out  (𝑆≈𝐾𝑚) , a large time step might introduce numerical error (e.g. leading to a negative substrate concentration).

**TMI:** scipy.integrate.odeint is a python wrapper around the FORTRAN function LSODA, which automatically switches between stiff and non-stiff solvers based on evaluation of eigenvalues of the Jacobian matrix. You don't get a choice in what solver gets used. If you want more control (perhaps you know something more about the behavior of your system), you can use **scipy.integrate.solve_ivp** or **scipy.integrate.ode** instead, which allows to explicitly choose the solver you want to use. solve_ivp is newer, but at the moment is slower (at least for problems I've tried) and is less developed than odeint.

First, we import the packages we need:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
#from scipy.integrate import solve_ivp (If you wanted to use solve_ivp instead)
%matplotlib inline

Next, we'll define the variables we will need

In [ ]:
FA0 = 0.1362 #mol/s
k = 0.0074 #mol/kg-s (k' in text)
alpha = 0.0367 #1/kg
epsilon = -0.15

Define a function that returns the relevant derivatives (dX/dW and dp/dW) given current values of the dependent (X,p) and indepdent (W) variables

In [ ]:
def deriv(current, W): #Note that we explicitly made this function of W, even though it is not used. This is required by the odeint
    X = current[0]
    p = current[1]
    ra = -1.0*k*(1.0-X)/(1+epsilon*X)*p
    dXdW = -ra/FA0
    dpdW = -alpha*(1.0+epsilon*X)/(2.0*p)
    return dXdW,dpdW

Define the range of W over which we want to integrate, and the initial conditions

In [ ]:
W_range = np.linspace(0, 28, 50) #Based on problem statement
X0 = 0 #No conversion at beginning of reactor
p0 = 1.0 #Remember that p = P/P0
init = [X0, p0] #Define a list of our initial conditions

Now we call odeint to integrate our function.

In [ ]:
#Now the magic: call odeint on our defined problem
sol = odeint(deriv, init, W_range)

**TMI:** There are a lot of additional options to make odeint more functional. We can specificy the maximum and minimum stepsize, set maximal tolerances, as well as even highlight specific ranges of t (W here) where we know the algorithm should be extra careful while integrating. Full documentation here: https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html

In [ ]:
X_odeint = sol[:,0]
P_odeint = sol[:,1]
plt.plot(W_range, X_odeint, '*', label='X')
plt.plot(W_range, P_odeint, '*', label='p (P/P0)')
#We'll show a faster way of doing the above four lines below
plt.title('X vs W')
plt.xlabel('W (kg)')
plt.ylabel('p or X')
plt.legend()

In [ ]:
#Sometimes things go wrong with integration, and it's useful to track all output from the algorithm:
sol2, info = odeint(deriv, init, W_range, full_output=1)

In [ ]:
#For example, since this is an adapative method, we could look at the actual W points that where values were calculated
W_actual=info['tcur'] #Extract the dictionary 'tcur' from the aditional odeint data
y = [0.2]*len(W_actual) #Generate a list of y's to add to plot to show actual timesteps

In [ ]:
X_odeint = sol[:,0]
P_odeint = sol[:,1]
plt.plot(W_range, X_odeint, '*', label='X')
plt.plot(W_range, P_odeint, '*', label='p (P/P0)')
plt.title('X vs W')
plt.xlabel('W (kg)')
plt.ylabel('p or X')
#Up to this point is same as before, now adding 
plt.plot(W_actual, y, 'o', label = 'W')
plt.legend()

In [ ]:
#Exploring other data returned by odeint
#dictionaries: tsw, nst, nfe, etc.
print(info['nfe']) #This is the cumulative number of function evaluations

In [ ]:
#A faster way to plot each of the trajacetories that we solved:
np_sol = np.asarray(sol)
labels = ['X', 'p']
for col in range(np_sol.shape[1]): 
    plt.plot(W_range, np_sol[:,col], label=labels[col])
plt.legend()

Let's look at the above code in a little more detail, since at first glance it's a little confusing. We make use of three functions here:

1) **np.asarray()** converts a standard python array to a numpy array, a data type that has some very useful functions associated with it

2) **array.shape()** returns the size of an array in each dimension. For a two-dimensional array, it returns the list [rows, columns]. By specifically referring to shape[1], we are returning the number of columns in the array

3) **range(start,stop,step)** returns a list of numbers from start position to stop position incremented by step. Only STOP is a required argument. By default, if only STOP is given, this function returns a list from 0->stop in increments of 1. Note that because of Python's 0-indexing, position 2 is index 1, so range(2), gives us 0,1

so, combining range(np_sol.shape[1]) gives us the list 0,1. 

The for loop we set up iterates over this range, with the variable col taking on each value. We then use that value to extract the specific column and plot it, with the associated label. The code below demonstrates the various aspects discussed here.

In [ ]:
np_sol.shape
#50 rows, 2 columns

In [ ]:
range(2)
for n in range(2):
  print(n)

## A word about variable access
In this example, we defined alpha, FA0, and other variables **globally** (outside of a specific function), e.g. they can be accessed in any function we write. This *can* be really dangerous, depending on the order cells are run within a notebook, and can have unintended consequences. Sometimes it is better to keep variables defined **locally** e.g. within a function, and then pass them off to another function when it is called. For most user-defined functions, this is straightforward, e.g.:

In [ ]:
def execute():
    x = np.linspace(0,10,11)
    x2 = multiply_by_2(x)
    print(x2)
    
def multiply_by_2(x):
    return 2.0*x

execute() #Run the function loop_over_x and print the output

In [ ]:
#But now if I try to access x
try:
    print(x)
except Exception as e:
    print("Oops, that caused an error because ", str(e))

This is a **good thing**, because you can't accidentally change the definition of x in "execute" in another function that you write. 

One annoying exception to this simplicity is when using odeint (and other built-in functions) to pass things like FA0 and alpha to your derivative function. Let's take the toy example:
$$\frac{dX}{dt}=AX^b$$

In [ ]:
def deriv(X,t,a,b):
    #If I want the calling function to control a and b, I can't define them here!
    dXdt = a*X**b
    return dXdt

In [ ]:
def run_solver(a,b):
    X0 = 1.0
    t_range = np.linspace(0, 3.0, 50)
    sol = odeint(deriv,X0, t_range, args=(a,b)) #Note the syntax and use of "args" here. The parentheses indicate these values are tuples (immutable)
    plt.plot(t_range,sol[:,0])

for a in np.linspace(0.4, 1.0, 3):
    run_solver(a,1.0)

    #This is a cleaner way to input variables into your derivative function